# Install `simex`

In [1]:
%%bash
#clone repo of `sipecam-metadata-extactor` in server
#git clone git@github.com:CONABIO/sipecam-metadata-extractor.git ~/
cd ../src
#install in editable mode
pip install -q -e .

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# List of files to extract metadata

In [1]:
%%bash
dir_with_sipecam_data=/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM
list_of_files_and_subdirectories_to_extract_metadata --input_directory $dir_with_sipecam_data

## Check

In [2]:
%%bash
ls -lh /shared_volume/sipecam_files_to_extract_metadata_from_*

-rw-r--r-- 1 root root 3.2M Dec 16 18:03 /shared_volume/sipecam_files_to_extract_metadata_from_16-12-2021.txt


In [3]:
%%bash
ls -lh /shared_volume/sipecam_subdirectories_*

-rw-r--r-- 1 root root 1.8K Dec 16 18:03 /shared_volume/sipecam_subdirectories_16-12-2021.txt


In [4]:
%%bash
wc -l /shared_volume/sipecam_files_to_extract_metadata_from_*

29079 /shared_volume/sipecam_files_to_extract_metadata_from_16-12-2021.txt


In [5]:
%%bash
wc -l /shared_volume/sipecam_subdirectories_*

20 /shared_volume/sipecam_subdirectories_16-12-2021.txt


In [6]:
%%bash
#check every first subdirectory has a JPG, WAV or AVI file
SAVEIFS=$IFS;IFS=$(echo -en "\n\b")
for f in $(cat /shared_volume/sipecam_subdirectories_*.txt)
  do echo "$f"
     ls "$f"|grep "[WAV|wav|JPG|jpg|AVI|avi]$"|head -n 1
done
IFS=$SAVEIFS

/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico
20210728_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Audible
20210728_093000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_2/Ultrasonico
20210728_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_2/Audible
20210728_105000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_3/Nueva carpeta/Unidad de USB
20210728_141000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Ultrasonico
20210729_102000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible
20210729_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_4/Audible
20210729_133000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_1/Audio

# Generate SiPeCaM Zendro schema (one time only)

In [1]:
%%bash
echo "SIPECAM_ZENDRO_GQL_URL=https://gql.sipecamdata.conabio.gob.mx/" > ~/.simex_env
echo "SIPECAM_ZENDRO_GQL_USER=<user>" >> ~/.simex_env
echo "SIPECAM_ZENDRO_GQL_PASSWORD=<password>" >> ~/.simex_env


In [10]:
%%bash
generate_sipecam_zendro_schema

Standard Error sgqlc.introspection b''
Standard Output sgqlc.introspection b'\n'
Standard Error sgqlc-codegen b''
Standard Output sgqlc-codegen b''


## Check

In [11]:
%%bash
head -n 10 /root/sipecam-metadata-extractor/src/simex/sipecam_zendro_schema.py

import sgqlc.types
import sgqlc.types.datetime


sipecam_zendro_schema = sgqlc.types.Schema()



########################################################################
# Scalars and Enumerations


# Extract serial number, datetime and coordinates

In [1]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/"
#default for parallel execution are 4 processes
extract_serial_number_datetime_and_coordinates_of_files --input_dir "$dir_to_be_processed" --parallel

In [2]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico"
#can select number of proceses
extract_serial_number_datetime_and_coordinates_of_files --input_dir "$dir_to_be_processed" --parallel --number_of_processes 6

In [3]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible"
#if number_of_processes is passed as arg then is assumed will execute in parallel
extract_serial_number_datetime_and_coordinates_of_files --input_dir "$dir_to_be_processed" --number_of_processes 6

## Example of dir with different serial for images

In [4]:
%%bash
dir_to_be_processed="/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata"
extract_serial_number_datetime_and_coordinates_of_files --input_dir "$dir_to_be_processed" --mixed

## Check

### Dir with image files

In [5]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_datetime_and_coordinates.logs")
head -n 15 "$file_for_logs"

2022-02-05 13:21:09,142 [INFO]  extraction of serial number, datetime and coordinates
2022-02-05 13:21:09,143 [INFO]  logs for extraction of serial number, datetime and coordinates in /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/100RECNX_simex_extract_serial_numbers_datetime_and_coordinates.json
2022-02-05 13:21:09,148 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2022-02-05 13:21:09,351 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2022-02-05 13:21:12,222 [INFO]  extraction of datetime of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2022-02-05 13:21:12,223 [INFO]  SUCCESSFUL extraction of datetime of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338

In [6]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/"
file_json=$(find "$dir_to_be_processed" -name "*.json")
python3 -mjson.tool "$file_json"|head -n 15

{
    "SerialNumber": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG": "HLPXGM09048890"
    },
    "Coordinates": {},
    "Datetime": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0004.JPG": "2021-07-28",
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0074.JPG": "2021-08-26"
    },
    "DaysBetweenFirstAndLastDatetime": 29
}


### Dir with audio files

In [7]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_datetime_and_coordinates.logs")
head -n 15 "$file_for_logs"

2022-02-05 13:21:12,570 [INFO]  extraction of serial number, datetime and coordinates
2022-02-05 13:21:12,572 [INFO]  logs for extraction of serial number, datetime and coordinates in /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/Ultrasonico_simex_extract_serial_numbers_datetime_and_coordinates.json
2022-02-05 13:21:12,579 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2022-02-05 13:21:12,785 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2022-02-05 13:22:06,799 [INFO]  extraction of datetime of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2022-02-05 13:22:06,799 [INFO]  SUCCESSFUL extraction of datetime of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octu

In [8]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico"
file_json=$(find "$dir_to_be_processed" -name "*.json")
python3 -mjson.tool "$file_json" | head -n 15

{
    "SerialNumber": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV": "2423C2045F256D74"
    },
    "Coordinates": {},
    "Datetime": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210728_212000.WAV": "2021-07-28",
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210826_020000.WAV": "2021-08-26"
    },
    "DaysBetweenFirstAndLastDatetime": 29
}


### Dir with audio files

In [9]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_datetime_and_coordinates.logs")
head -n 15 "$file_for_logs"

2022-02-05 13:22:07,324 [INFO]  extraction of serial number, datetime and coordinates
2022-02-05 13:22:07,326 [INFO]  logs for extraction of serial number, datetime and coordinates in /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/Audible_simex_extract_serial_numbers_datetime_and_coordinates.json
2022-02-05 13:22:07,334 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV
2022-02-05 13:22:07,542 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV
2022-02-05 13:23:03,628 [INFO]  extraction of datetime of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV
2022-02-05 13:23:03,628 [INFO]  SUCCESSFUL extraction of datetime of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM

In [10]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible"
file_json=$(find "$dir_to_be_processed" -name '*.json')
python3 -mjson.tool "$file_json" | head -n 15

{
    "SerialNumber": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV": "248D9B045EC9E6B1"
    },
    "Coordinates": {},
    "Datetime": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210729_220000.WAV": "2021-07-29",
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210828_004000.WAV": "2021-08-28"
    },
    "DaysBetweenFirstAndLastDatetime": 30
}


### Dir with different serial for images

In [11]:
%%bash
dir_to_be_processed="/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_datetime_and_coordinates.logs")
head -n 15 "$file_for_logs"

2022-02-05 13:23:04,155 [INFO]  extraction of serial number, datetime and coordinates
2022-02-05 13:23:04,156 [INFO]  logs for extraction of serial number, datetime and coordinates in /LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/read_image_metadata_simex_extract_serial_numbers_datetime_and_coordinates.json
2022-02-05 13:23:04,161 [INFO]  extraction of serial number of /LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX0001.JPG
2022-02-05 13:23:04,364 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX0001.JPG
2022-02-05 13:23:04,364 [INFO]  extraction of serial number of /LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX1736.JPG
2022-02-05 13:23:04,573 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks

In [12]:
%%bash
dir_to_be_processed="/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata"
file_json=$(find "$dir_to_be_processed" -name '*.json')
echo $file_json
python3 -mjson.tool "$file_json"

/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/read_image_metadata_simex_extract_serial_numbers_datetime_and_coordinates.json
{
    "SerialNumber": {
        "/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX0001.JPG": "HLPXGM09048701",
        "/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX1736.JPG": "HLPXGM09048480"
    },
    "Coordinates": {},
    "Datetime": {
        "/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX0001.JPG": "2021-10-26",
        "/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata/RCNX1736.JPG": "2021-08-17"
    }
}


# Extract metadata and ingest it

In [12]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [13]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [14]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

## Check

In [15]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:34,559 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2021-12-15 11:35:34,561 [INFO]  Read metadata of JPG
2021-12-15 11:35:34,561 [INFO]  {}
2021-12-15 11:35:34,561 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:34,562 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/JPG


In [16]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:35,463 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI
2021-12-15 11:35:35,465 [INFO]  Read metadata of AVI
2021-12-15 11:35:35,465 [INFO]  {}
2021-12-15 11:35:35,465 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:35,465 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/AVI


In [17]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:35,910 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2021-12-15 11:35:35,912 [INFO]  Read metadata of WAV
2021-12-15 11:35:35,912 [INFO]  {}
2021-12-15 11:35:35,912 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:35,912 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/WAV


# Notes:

One option:

Serial number + coordinates of image -> node, cumulus, date of deployment (all this from Zendro models)

but for audio we can't retrieve coordinates of recording.

Another option:

Serial number + date of file (audio, image, video) -> node, cumulus and date of deployment from deployment model of Zendro

